In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('/Users/younghun/Desktop/gitrepo/data')

In [2]:
data = pd.read_csv('kangwon.csv', encoding='utf-8')
data.shape

(1230355, 13)

In [3]:
# 필요없는 칼럼 삭제
del data['Unnamed: 0']
data.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,Datetime
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019-01
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019-01
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019-01
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019-01
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019-01


In [4]:
# 결측치 확인
data.isnull().sum()

CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
Datetime        0
dtype: int64

In [5]:
# 시군구도 필요없으니 시군구 변수도 삭제
del data['CARD_CCG_NM']

In [6]:
# 칼럼명 변경
cols = ['가맹점주소', '가맹점업종', '사용자거주시도', '사용자거주구군', '연령대',
       '성별', '가구생애주기', '카드사용인원수(환불사용자포함)', '카드사용금액', '카드사용건수(환불사용자제외)',
       '날짜']
data.columns = cols

In [7]:
# 카드사용금액을 맨 마지막 칼럼 순서로 돌리기
col_list = data.columns.tolist()
col_list.remove('날짜')
col_list.insert(0,'날짜')
col_list.remove('카드사용금액')
col_list.insert(11,'카드사용금액')

In [8]:
data = data[col_list]
data.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,건강보조식품 소매업,강원,강릉시,20s,1,1,4,4,311200
1,2019-01,강원,건강보조식품 소매업,강원,강릉시,30s,1,2,7,8,1374500
2,2019-01,강원,건강보조식품 소매업,강원,강릉시,30s,2,2,6,6,818700
3,2019-01,강원,건강보조식품 소매업,강원,강릉시,40s,1,3,4,5,1717000
4,2019-01,강원,건강보조식품 소매업,강원,강릉시,40s,1,4,3,3,1047300


In [9]:
data.describe()

,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
count,1.230355e+06,1.230355e+06,1.230355e+06,1.230355e+06,1.230355e+06
mean,1.379023e+00,2.852438e+00,2.879178e+01,5.232842e+01,1.291994e+06
std,4.851441e-01,1.354864e+00,2.037824e+02,4.719245e+02,1.047847e+07
min,1.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,1.820000e+03
25%,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,7.930000e+04
50%,1.000000e+00,3.000000e+00,6.000000e+00,8.000000e+00,1.965000e+05
75%,2.000000e+00,4.000000e+00,1.400000e+01,2.000000e+01,5.211000e+05
max,2.000000e+00,5.000000e+00,1.396900e+04,4.875400e+04,8.586119e+08


In [10]:
# 업종 종류 개수 살펴보기
data['가맹점업종'].nunique()

40

In [11]:
# 업종별로 카드사용건수 합계 집계해보기
upjong_cnt = data.groupby('가맹점업종').agg({'카드사용건수(환불사용자제외)':'sum'}).rename(columns={'카드사용건수(환불사용자제외)':'카드사용건수합계'})
upjong_cnt.head()

,카드사용건수합계
가맹점업종,
건강보조식품 소매업,16170
골프장 운영업,465266
과실 및 채소 소매업,428427
관광 민예품 및 선물용품 소매업,22012
그외 기타 분류안된 오락관련 서비스업,10


In [12]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [13]:
upjong_cnt = upjong_cnt.sort_values(by='카드사용건수합계', ascending=False)
upjong_cnt.iplot(kind='bar', theme='white')

In [14]:
# 업종에 따른 카드사용금액 평균값으로 비교해보기
upjong_cnt = data.groupby('가맹점업종').agg({'카드사용금액':'mean'}).rename(columns={'카드사용금액':'카드사용금액_평균'})
upjong_cnt = upjong_cnt.sort_values(by='카드사용금액_평균', ascending=False)
upjong_cnt.iplot(kind='bar', theme='white')

In [15]:
# 카드사용금액 평균값이랑 카드사용건수 합계 두 개 기준으로 업종에 따라 오름차순으로 정렬
# 오름차순으로 정렬하는 이유는 추후에 업종별 순위로 Weight를 편하게 매기기 위함임
upjong  = data.groupby('가맹점업종').agg({'카드사용건수(환불사용자제외)':'sum','카드사용금액':'mean'}).rename(columns={'카드사용건수(환불사용자제외)':'카드사용건수합계',
                                                                                               '카드사용금액':'카드사용금액_평균'})
# sort_values 기준을 다량으로 부여할 때, 앞의 요소를 우선순위로 sort한다.
upjong = upjong.sort_values(by=['카드사용금액_평균','카드사용건수합계'], ascending=True)
upjong.head(20)

,카드사용건수합계,카드사용금액_평균
가맹점업종,,
택시 운송업,206,62926.428571
버스 운송업,8018,63660.787671
내항 여객 운송업,1571,87656.054187
그외 기타 분류안된 오락관련 서비스업,10,88500.000000
관광 민예품 및 선물용품 소매업,22012,103254.744354
여행사업,69465,148423.670231
기타 수상오락 서비스업,3698,176712.009804
비알콜 음료점업,2171921,288395.523733
면세점,88,344198.571429


In [19]:
pd.options.display.max_rows=41
upjong_ranking = upjong.index.tolist()
print(upjong_ranking)

['택시 운송업', '버스 운송업', '내항 여객 운송업', '그외 기타 분류안된 오락관련 서비스업', '관광 민예품 및 선물용품 소매업', '여행사업', '기타 수상오락 서비스업', '비알콜 음료점업', '면세점', '피자 햄버거 샌드위치 및 유사 음식점업', '전시 및 행사 대행업', '그외 기타 스포츠시설 운영업', '수산물 소매업', '빵 및 과자류 소매업', '욕탕업', '과실 및 채소 소매업', '여관업', '서양식 음식점업', '휴양콘도 운영업', '체인화 편의점', '자동차 임대업', '호텔업', '마사지업', '기타 외국식 음식점업', '중식 음식점업', '일식 음식점업', '차량용 가스 충전업', '건강보조식품 소매업', '슈퍼마켓', '화장품 및 방향제 소매업', '골프장 운영업', '한식 음식점업', '기타 대형 종합 소매업', '육류 소매업', '기타 주점업', '차량용 주유소 운영업', '기타음식료품위주종합소매업', '스포츠 및 레크레이션 용품 임대업', '그외 기타 종합 소매업', '일반유흥 주점업']


In [21]:
# dict형태로 만들어주기 위해서 업종별 옆에다가 가중치 점수 추가해주기
for weight in range(0, 40):
    upjong_ranking.insert(2*weight+1, weight+1)
print(upjong_ranking)

['택시 운송업', 1, '버스 운송업', 2, '내항 여객 운송업', 3, '그외 기타 분류안된 오락관련 서비스업', 4, '관광 민예품 및 선물용품 소매업', 5, '여행사업', 6, '기타 수상오락 서비스업', 7, '비알콜 음료점업', 8, '면세점', 9, '피자 햄버거 샌드위치 및 유사 음식점업', 10, '전시 및 행사 대행업', 11, '그외 기타 스포츠시설 운영업', 12, '수산물 소매업', 13, '빵 및 과자류 소매업', 14, '욕탕업', 15, '과실 및 채소 소매업', 16, '여관업', 17, '서양식 음식점업', 18, '휴양콘도 운영업', 19, '체인화 편의점', 20, '자동차 임대업', 21, '호텔업', 22, '마사지업', 23, '기타 외국식 음식점업', 24, '중식 음식점업', 25, '일식 음식점업', 26, '차량용 가스 충전업', 27, '건강보조식품 소매업', 28, '슈퍼마켓', 29, '화장품 및 방향제 소매업', 30, '골프장 운영업', 31, '한식 음식점업', 32, '기타 대형 종합 소매업', 33, '육류 소매업', 34, '기타 주점업', 35, '차량용 주유소 운영업', 36, '기타음식료품위주종합소매업', 37, '스포츠 및 레크레이션 용품 임대업', 38, '그외 기타 종합 소매업', 39, '일반유흥 주점업', 40]


In [22]:
# dict 형태로 변환해주는 함수 정의
def convert_to_dict(lst):
    dict_type = {lst[i] : lst[i+1] for i in range(0, len(lst), 2)}
    return dict_type

In [25]:
upjong_weight = convert_to_dict(upjong_ranking)
print(upjong_weight)

{'택시 운송업': 1, '버스 운송업': 2, '내항 여객 운송업': 3, '그외 기타 분류안된 오락관련 서비스업': 4, '관광 민예품 및 선물용품 소매업': 5, '여행사업': 6, '기타 수상오락 서비스업': 7, '비알콜 음료점업': 8, '면세점': 9, '피자 햄버거 샌드위치 및 유사 음식점업': 10, '전시 및 행사 대행업': 11, '그외 기타 스포츠시설 운영업': 12, '수산물 소매업': 13, '빵 및 과자류 소매업': 14, '욕탕업': 15, '과실 및 채소 소매업': 16, '여관업': 17, '서양식 음식점업': 18, '휴양콘도 운영업': 19, '체인화 편의점': 20, '자동차 임대업': 21, '호텔업': 22, '마사지업': 23, '기타 외국식 음식점업': 24, '중식 음식점업': 25, '일식 음식점업': 26, '차량용 가스 충전업': 27, '건강보조식품 소매업': 28, '슈퍼마켓': 29, '화장품 및 방향제 소매업': 30, '골프장 운영업': 31, '한식 음식점업': 32, '기타 대형 종합 소매업': 33, '육류 소매업': 34, '기타 주점업': 35, '차량용 주유소 운영업': 36, '기타음식료품위주종합소매업': 37, '스포츠 및 레크레이션 용품 임대업': 38, '그외 기타 종합 소매업': 39, '일반유흥 주점업': 40}


---

In [33]:
# 가중치 부여해주는 함수 되는지 test용
test = pd.DataFrame(
{'가맹점업종':['택시 운송업', '버스 운송업', '내항 여객 운송업'],
'지역':['강원','강원','강원']},
columns=['지역','가맹점업종'])
test

,지역,가맹점업종
0,강원,택시 운송업
1,강원,버스 운송업
2,강원,내항 여객 운송업


In [36]:
test.apply(give_to_weight, axis=1)

,지역,가맹점업종
0,강원,1
1,강원,2
2,강원,3


---

In [38]:
# upjong_weight를 이용해 원본 데이터에다가 업종별로 정해진 가중치 부여해주기
def give_to_weight(df):
    if df['가맹점업종'] in upjong_weight.keys():
        df['가맹점업종'] = upjong_weight[df['가맹점업종']]
    return df

In [39]:
data = data.apply(give_to_weight, axis=1)
data.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,28,강원,강릉시,20s,1,1,4,4,311200
1,2019-01,강원,28,강원,강릉시,30s,1,2,7,8,1374500
2,2019-01,강원,28,강원,강릉시,30s,2,2,6,6,818700
3,2019-01,강원,28,강원,강릉시,40s,1,3,4,5,1717000
4,2019-01,강원,28,강원,강릉시,40s,1,4,3,3,1047300


In [40]:
data.to_csv('./categorical_encoding.csv', encoding='utf-8')